#### Large Language Model Component
- Used for calling the large language model pipeline in Google Colab to use high-spec GPU for faster LLM inference
- To be stored in Google Colab as a python file/script
- Contains a duplicate of prompt generator's LLM related functions (but not iterative), and Modelfile Text for the Ollama Modelfile after Fine Tuning

##### Main LLM Script

In [ ]:
## Prompt Generator + LLM
# Langchain and Ollama
import langchain
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

## Settings
TEMPLATE_SETTING = 1
DEBUG_SETTING = 1

## Template for non-finetuned Llama 3.1 8B model. Very Consistent
if (TEMPLATE_SETTING == 0):
    template = """You are an expert in software quality assurance specializing in usability testing.

    Consider the following usability aspects:
    Accessibility (keyboard navigation, screen reader support, etc.)
    Responsiveness (behavior across different screen sizes, etc.)
    Feedback (hover effects, click responses, error messages, etc.)
    Interactivity (expected behavior when clicked, typed into, or focused, etc.)
    User experience (clarity of labels, ease of use, etc.)

    Avoid descriptions like "The user" or "The test participant" and should instead focus on direct, executable actions.

    Given this UI Element: {ui_element}
    From this website: {url}

    Generate a functional usability test case with the following aspects, separated by a `~`:
    - **Objective**: Clearly state the objective of the test.
    - **Preconditions**: List any preconditions that need to be met.
    - **Test Steps**: Provide a step-by-step guide for the test. Each step should be a direct command, not a description.
    - **Expected Output**: Describe the expected output.

    Output format example, start with the objective directly and avoid saying "here is the test case" or the name of the testcase and end the output after writing the expected output:
    Objective: [Describe the objective] 
    ~ Preconditions: [List preconditions, use dashes] 
    ~ Test Steps: [Step-by-step guide] 
    ~ Expected Output: [Describe the expected output, use dashes]"""

    model_str = "llama3.1"

## Template for fine-tuned Llama 3.1 8B model. Can be very sensitive to the prompt
# - Experimental
elif (TEMPLATE_SETTING == 1):
    template = """Generate a functional usability test case for the following UI element: {ui_element} from the website: {url}"""
    model_str = "qallama"

## Returns Model
# - Call using: chain.invoke({"ui_element": str(item), "url": url})
# - str(item) is string version of the scraped element
# - url is the website url
def load_model_chain(template : str =  template, model_str : str = model_str, temperature=0):
    prompt = ChatPromptTemplate.from_template(template)
    # For Non-Fine Tuned
    if TEMPLATE_SETTING == 0:
        model = OllamaLLM(model=model_str, temperature=temperature)
    else:
        model = OllamaLLM(model=model_str)
    chain = prompt | model
    return chain

##### Modelfile
- replace input content with the context